# Transport Network Metrics

## 0 Setup environment

In [ ]:
import os

import geopandas as gpd
import networkx as nx
import osmnx as ox
import pandas as pd
from networkx import Graph

In [3]:
BASE_PATH = "./data"
BOUNDARIES_FILE = os.path.join(BASE_PATH, "boundaries.gpkg")
TRANSPORT_NETWORK_FILE = os.path.join(BASE_PATH, "merged_network.graphml")

## 1 Load town boundaries

In [4]:
gdf_boundaries = gpd.read_file(BOUNDARIES_FILE)
gdf_boundaries.head()

,town,province,geometry
0,Bacacay,Albay,"MULTIPOLYGON (((123.84193 13.3341, 123.84204 1..."
1,Camalig,Albay,"MULTIPOLYGON (((123.6559 13.06131, 123.65536 1..."
2,Daraga,Albay,"MULTIPOLYGON (((123.71487 13.03995, 123.71474 ..."
3,Guinobatan,Albay,"MULTIPOLYGON (((123.68355 13.25321, 123.67729 ..."
4,Jovellar,Albay,"MULTIPOLYGON (((123.6559 13.06131, 123.65568 1..."


## 2 Load transport network

In [ ]:
graph_transport = ox.load_graphml(TRANSPORT_NETWORK_FILE)
gdf_nodes = ox.graph_to_gdfs(graph_transport, edges=False)
gdf_nodes.head()

,y,x,street_count,mode,highway,junction,railway,geometry
osmid,,,,,,,,
300744370,11.762615,124.033762,3,road,NaN,NaN,NaN,POINT (124.03376 11.76261)
300744933,11.764680,124.063952,3,road,NaN,NaN,NaN,POINT (124.06395 11.76468)
300744970,11.863829,124.057783,3,road,NaN,NaN,NaN,POINT (124.05778 11.86383)
300746507,11.964546,123.991640,3,road,NaN,NaN,NaN,POINT (123.99164 11.96455)
300747572,11.851925,124.060147,3,road,NaN,NaN,NaN,POINT (124.06015 11.85192)


## 3 Compute centrality measures

In [15]:
gdf_nodes["degree"] = nx.degree_centrality(graph_transport).values()
gdf_nodes["degree"].describe()

count    56401.000000
mean         0.000044
std          0.000018
min          0.000000
25%          0.000018
50%          0.000053
75%          0.000053
max          0.000142
Name: degree, dtype: float64

In [ ]:
gdf_nodes["closeness"] = nx.closeness_centrality(graph_transport).values()
gdf_nodes["closeness"].describe()

count    56401.000000
mean         0.004990
std          0.001106
min          0.000000
25%          0.004116
50%          0.005104
75%          0.005874
max          0.006919
Name: closeness, dtype: float64

In [22]:
gdf_nodes["betweenness"] = nx.betweenness_centrality(graph_transport).values()
gdf_nodes["betweenness"].describe()

count    56401.000000
mean         0.003528
std          0.016094
min          0.000000
25%          0.000000
50%          0.000070
75%          0.000456
max          0.300338
Name: betweenness, dtype: float64

In [23]:
filepath = os.path.join(BASE_PATH, "centrality_measures_2.csv")
gdf_nodes.to_csv(filepath, columns=["degree", "closeness", "betweenness"], index=True)

## 4 Compute metrics for each town

In [ ]:
def calculate_town_metrics(
    town_name: str,
    boundaries_gdf: gpd.GeoDataFrame,
    nodes_gdf: gpd.GeoDataFrame,
    full_graph: nx.Graph,
) -> dict:
    """Calculates all transport network metrics for a single town."""
    town_metrics = {"town": town_name}
    town_boundary = boundaries_gdf[boundaries_gdf["town"] == town_name]

    nodes_in_town = gpd.sjoin(nodes_gdf, town_boundary, how="inner", predicate="within")
    town_metrics["avg_degree"] = nodes_in_town["degree"].mean()
    town_metrics["avg_betweenness"] = nodes_in_town["betweenness"].mean()
    town_metrics["avg_closeness"] = nodes_in_town["closeness"].mean()

    town_subgraph = full_graph.subgraph(nodes_in_town.index)

    if not nx.is_connected(town_subgraph):
        largest_cc = max(nx.connected_components(town_subgraph), key=len)
        subgraph_for_analysis = town_subgraph.subgraph(largest_cc)
    else:
        subgraph_for_analysis = town_subgraph

    town_metrics["network_density"] = nx.density(town_subgraph)
    town_metrics["avg_clustering"] = nx.average_clustering(Graph(town_subgraph))
    # town_metrics["network_diameter"] = nx.diameter(subgraph_for_analysis)
    # town_metrics["avg_path_length"] = nx.average_shortest_path_length(subgraph_for_analysis)

    return town_metrics

In [ ]:
all_town_metrics = [
    calculate_town_metrics(town_name, gdf_boundaries, gdf_nodes.copy(), graph_transport)
    for town_name in sorted(gdf_boundaries["town"].unique())
]

df_town_metrics = pd.DataFrame(all_town_metrics).set_index("town")
df_town_metrics.head()

                  avg_degree  avg_betweenness  avg_closeness  network_density  avg_clustering
town                                                                                         
Aroroy              0.000043     3.196802e-03       0.005466         0.003937        0.042272
Baao                0.000044     2.216463e-03       0.005905         0.004325        0.025000
Bacacay             0.000042     9.642322e-04       0.004373         0.004257        0.023723
Bagamanoc           0.000043     9.105469e-04       0.003144         0.014739        0.056911
Balatan             0.000046     5.637181e-03       0.006357         0.013931        0.038251
Baleno              0.000041     7.429846e-04       0.005049         0.017684        0.028424
Balud               0.000041     4.737773e-04       0.003407         0.007683        0.052013
Baras               0.000040     8.952088e-04       0.003124         0.009873        0.042754
Barcelona           0.000044     8.430132e-04       0.004171

In [ ]:
filepath = os.path.join(BASE_PATH, "town_metrics.csv")
df_town_metrics.to_csv(filepath, index=True)